## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"
- remember to restart runtime after installing deeplabcut


In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 593 kB 32.3 MB/s 
     |████████████████████████████████| 9.4 MB 56.5 MB/s 
     |████████████████████████████████| 296 kB 65.9 MB/s 
     |████████████████████████████████| 352 kB 82.1 MB/s 
     |████████████████████████████████| 109 kB 80.5 MB/s 
     |████████████████████████████████| 177 kB 85.4 MB/s 
     |████████████████████████████████| 965 kB 66.9 MB/s 
     |████████████████████████████████| 295 kB 77.0 MB/s 
     |████████████████████████████████| 555 kB 61.7 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=a45d39353f08911a2097f07e049a944ff9324d8138688a2c59163a20d6377b7c
  Stored in directory: /root/.cache/pip/wheels/fe/f6/cb/40331472edf4fd399b8cad02973c6acbdf26898342928327fe
Successfully built filterpy
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    

#install other packages and setup
- connect to google drive in this step

In [8]:
#a few colab specific things needed:
!pip install --upgrade scikit-image
!pip3 install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import os
import deeplabcut
import numpy as np
import pandas as pd
import pickle5 as pickle
from deeplabcut.utils import auxfun_multianimal, auxiliaryfunctions
import random
from pathlib import Path
import glob

Link your Google Drive (with your labeled data):

In [10]:
#Now, let's link to your Google Drive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [11]:
# # Download our demo project:
# import urllib.request
# from io import BytesIO
# from zipfile import ZipFile
# 
# def unzip_from_url(url, dest_folder='/'):
#    # Directly extract files without writing the archive to disk
#    resp = urllib.request.urlopen(url)
#    with ZipFile(BytesIO(resp.read())) as zf:
#       zf.extractall(path=dest_folder)

# project_url = "http://deeplabcut.rowland.harvard.edu/datasets/demo-me-2021-07-14.zip"
# unzip_from_url(project_url, "/content/drive/MyDrive/")

In [12]:
project_path = "/content/drive/MyDrive/demo-me-2021-07-14"
config_path = os.path.join(project_path, "config.yaml")
tracklets_config_path = os.path.join(project_path, "tracklets_config.yaml")

In [13]:
TRACK_METHOD = "ellipse"  # Could also be "box", but "ellipse" was found to be more robust on this dataset.

In [14]:
!pip show deeplabcut

Name: deeplabcut
Version: 2.2.3
Summary: Markerless pose-estimation of user-defined features with deep learning
Home-page: https://github.com/DeepLabCut/DeepLabCut
Author: A. & M. Mathis Labs
Author-email: alexander@deeplabcut.org
License: None
Location: /usr/local/lib/python3.8/dist-packages
Requires: scipy, pyyaml, tables, numba, filterpy, tensorpack, numpy, tensorflow, networkx, ruamel.yaml, imgaug, scikit-learn, scikit-image, tf-slim, matplotlib, pandas, Pillow, tqdm, statsmodels
Required-by: 


# _running analysis for 221127 right now_

#Analyze all videos in "videos" folder:

In [16]:
#running for 221127 right now

#for video in [project_path + "/videos/PZ71_1.avi"]:
for video in glob.glob(project_path + "/videos/" + "*.avi"):
  deeplabcut.analyze_videos(config_path,[video], shuffle=0, videotype="avi",auto_track=False )
  deeplabcut.convert_detections2tracklets(
      config_path,
      [video],
      videotype='avi',
      shuffle=0,
      track_method=TRACK_METHOD,
      ignore_bodyparts=["tail1", "tail2", "tailend"],  # Some body parts can optionally be ignored during tracking for better assembly (but they are used later)
  )
  deeplabcut.stitch_tracklets(
    config_path,
    [video],
    videotype='avi',
    shuffle=0,
    track_method=TRACK_METHOD,
    n_tracks=3,
  )
  
  #Filter the predictions to remove small jitter, if desired:
  deeplabcut.filterpredictions(config_path, 
                                [video], 
                                shuffle=0,
                                videotype='avi', 
                                track_method = TRACK_METHOD)
  deeplabcut.create_labeled_video(
    config_path,
    [video],
    videotype='avi',
    shuffle=0,
    color_by="individual",
    keypoints_only=False,
    draw_skeleton=True,
    filtered=True,
    track_method=TRACK_METHOD,
  )

Using snapshot-20000 for model /content/drive/MyDrive/demo-me-2021-07-14/dlc-models/iteration-0/demoJul14-trainset95shuffle0
Activating extracting of PAFs
Starting to analyze %  /content/drive/MyDrive/demo-me-2021-07-14/videos/PZ70_1.avi
Video already analyzed! /content/drive/MyDrive/demo-me-2021-07-14/videos/PZ70_1DLC_dlcrnetms5_demoJul14shuffle0_20000.h5
The videos are analyzed. Time to assemble animals and track 'em... 
 Call 'create_video_with_all_detections' to check multi-animal detection quality before tracking.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Using snapshot-20000 for model /content/drive/MyDrive/demo-me-2021-07-14/dlc-models/iteration-0/demoJul14-trainset95shuffle0
Processing...  /content/drive/MyDrive/demo-me-2021-07-14/videos/PZ70_1.avi
Tracklets already computed /content/drive/MyDrive/demo-me-2021-07-14/videos/PZ70_1DLC_d

100%|██████████| 212/212 [00:00<00:00, 2309.71it/s]


Filtering with median model /content/drive/MyDrive/demo-me-2021-07-14/videos/PZ70_1.avi
Data from PZ70_1 were already filtered. Skipping...
Starting to process video: /content/drive/MyDrive/demo-me-2021-07-14/videos/PZ70_1.avi
Loading /content/drive/MyDrive/demo-me-2021-07-14/videos/PZ70_1.avi and data.
Labeled video already created. Skipping...
Using snapshot-20000 for model /content/drive/MyDrive/demo-me-2021-07-14/dlc-models/iteration-0/demoJul14-trainset95shuffle0
Activating extracting of PAFs
Starting to analyze %  /content/drive/MyDrive/demo-me-2021-07-14/videos/PZ71_1.avi
Video already analyzed! /content/drive/MyDrive/demo-me-2021-07-14/videos/PZ71_1DLC_dlcrnetms5_demoJul14shuffle0_20000.h5
The videos are analyzed. Time to assemble animals and track 'em... 
 Call 'create_video_with_all_detections' to check multi-animal detection quality before tracking.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_out

100%|██████████| 307/307 [00:02<00:00, 153.45it/s]


ValueError: ignored